In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [47]:
import numpy as np
import pandas as pd
from kcsd import KCSD1D

In [48]:
from ecephys.sglx_utils import load_timeseries
from ecephys_analyses.data import channel_groups, paths
from ecephys.plot import lfp_explorer, colormesh_timeseries_explorer
from ecephys.signal.utils import mean_subtract, median_subtract
from ecephys.signal.csd import get_kcsd

In [49]:
SUBJECT = "Doppio"
START_TIME = 2123 # pick a 1 second window that has SPW-Rs

In [50]:
chans = channel_groups.full[SUBJECT]
bin_path = paths.get_sglx_style_datapaths(subject=SUBJECT, experiment="sleep-homeostasis", condition="recovery-sleep", ext="lf.bin")[0]
_sig = load_timeseries(bin_path, chans, start_time=(START_TIME), end_time=(START_TIME + 1.0))
(sig, times, fs) = (_sig.values, _sig.time.values, _sig.fs)

nChan: 385, nFileSamp: 18000019


In [51]:
# sig = mean_subtract(sig)

## 1D kCSD on whole probe

In [52]:
gdx = intersite_distance = 0.020
k = get_kcsd(
    sig, intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
)
est_csd = k.values('CSD')

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-09
max lambda 0.0043
min lambda 1e-09
max lambda 0.0043
l-curve (all lambda):  0.23
Best lambda and R =  0.0018628719929130466 ,  0.23


In [62]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_x, est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

lo = 5.7333
hi = 6.0666
ax.axhline(hi, alpha=0.5, color='k', linestyle="--")
ax.axhline(lo, alpha=0.5, color='k', linestyle="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
hpc_chans = chans[np.logical_and(k.ele_pos >= lo, k.ele_pos <= hi).flatten()] # These are the channel numbers belonging to the hippocampus
print(hpc_chans)

[190 193 194 197 198 201 202 205 206 209 210 213 214 217 218 221 222]


In [42]:
hpc_idx = np.where(np.logical_and(k.ele_pos >= lo, k.ele_pos <= hi).flatten())[0] # These are the indices of the above channels

In [43]:
assert np.array_equal(hpc_chans, chans[hpc_idx]) # Confirm that indexing is correct

In [44]:
print(f"Min HPC index: {hpc_idx.min()} \nMax HPC index: {hpc_idx.max() + 1}")

Min HPC index: 175 
Max HPC index: 291


## 1D kCSD on hippocampal channels

In [251]:
hpc_k = get_kcsd(
    sig[:, hpc_idx], intersite_distance=intersite_distance, gdx=gdx, do_lcurve=True
)
hpc_est_csd = hpc_k.values('CSD')

Performing L-curve parameter estimation...
No lambda given, using defaults
min lambda 1e-12
max lambda 0.0119
min lambda 1e-12
max lambda 0.0119
l-curve (all lambda):  0.23
Best lambda and R =  2.8175859627735274e-05 ,  0.23


In [252]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, hpc_k.estm_x, hpc_est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

# Visualize stratum radiatum position
sr_chans = channel_groups.stratum_radiatum_140um_to_200um[SUBJECT]
sr_mask = np.isin(hpc_chans, sr_chans)
sr_pos = hpc_k.ele_pos[sr_mask]
ax.axhline(sr_pos.min(), alpha=0.5, color='k', linestyle=":")
ax.axhline(sr_pos.max(), alpha=0.5, color='k', linestyle=":")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …